In [1]:
from sklearn import svm, metrics
import glob, os.path, re, json

# re: 정규표현식
# 파일 저장 시 json 형태로 저장하기 위해 json import

#### Code 해석 순서
1. main 부터 찾아서 해석
2. 함수 선언은 main 위쪽 ( 호출이 빠른 순서대로 main과 가까운 위치 )

In [2]:
def check_freq(fname):
#     file open + 음절계산
    
    with open(fname, 'r', encoding='utf-8') as f:
#         readLines() 쓰지 않은 이유: 음절 단위로 끊으므로 줄바꿈 고려X
        text = f.read()        
        pass
    text = text.lower()
    
    cnt = [ 0 for n in range(0,26) ]
#     메모리 비효율성( 문자 -> 숫자 -> 문자 ) 해결 위해 Ascii 코드 변환
    code_a = ord('a')
    code_z = ord('z')
    
    for ch in text:
        n = ord(ch)
        if code_a <= n <= code_z:
            cnt[ n - code_a ] += 1
            pass
        pass
    
#     정규화하기 ( text간 상대 비교하기 위해 )
    total = sum(cnt)
    freq = list( map( lambda x: x / total, cnt ) )
    
#     lang????
    name = os.path.basename(fname)
#     r: unicode 방식에 맞춰 매핑(즉, 한글도 포함)
#     ^: 시작점부터 // group(): match 결과 반환
    lang = re.match(r'^[a-z]{2,}', name).group()
        
    return freq, lang
    


def load_files(path):
    freqs = []
    labels = []
    file_list = glob.glob(path)
    for fname in file_list:
        r = check_freq(fname)
        freqs.append(r[0])
        labels.append(r[1])
        pass
    # 전처리된 dataset을 json 형태로 저장하기 위한 data 구조
    return {'freqs': freqs, 'labels': labels}


##### main 시작

data = load_files('./lang/train/*.txt')
test = load_files('./lang/test/*.txt')

# 전처리된 dataset을 json 형태로 저장
with open('./lang/freq.json', 'w', encoding = 'utf-8') as fp:
    json.dump([data, test],fp)
    pass

### 모델 학습